# Welcome to the Modular Multimodal Data Fusion ML Pipeline for stress detection for the WESAD Database

## Table of contents






## Getting Started:
First, download necessary packages, if you are using a venv such as Conda, activate this first.

In [ ]:
%pip install -r requirements.txt

## Data Installation
If you are on Linux, run this cell to download and extract the WESAD dataset automatically, otherwise download manually [here](https://uni-siegen.sciebo.de/s/HGdUkoNlW1Ub0Gx/download) and unzip the `WESAD` file into the `wesad` directory i.e. `wesad/WESAD/`

In [ ]:
%cd src/wesad && bash download_database.sh
%cd ../..

## Data Preprocessing

This will automatically extract the biosensor data from the WESAD directory into several merged files in `.pkl` format.

This will take around 10 minutes depending on the machine.

In [ ]:
from src.wesad.data_preprocessing import WESADDataPreprocessor

preprocessor = WESADDataPreprocessor('src/wesad/WESAD/')
preprocessor.preprocess()

## Signal Preprocessing Steps
We will preprocess each signal with their respective preprocessing steps:

### Chest Signals

#### ECG
- **Smoothing**: Savitzky–Golay filter with window size 11 and order 3.
- **Filtering**: Butterworth band-pass filter of order 3 with cutoff frequencies 0.7 Hz and 3.7 Hz.

#### EMG
- **Smoothing**: Savitzky–Golay filter with window size 11 and order 3.
- **Filtering**: Butterworth lowpass filter of order 3 with cutoff frequency 0.5 Hz.

#### EDA
- **Smoothing**: Savitzky–Golay filter with window size 11 and order 3.
- **Filtering**: Butterworth lowpass filter of order 2 with cutoff frequency 5 Hz.

#### TEMP
- **Smoothing**: Savitzky–Golay filter with window size 11 and order 3.

#### RESP
- **Smoothing**: Savitzky–Golay filter with window size 11 and order 3.
- **Filtering**: Butterworth band-pass filter of order 3 with cutoff frequencies 0.1 Hz and 0.35 Hz.

#### ACC
- **Smoothing**: Savitzky–Golay filter with window size 31 and order 5.

### Wrist Signals

#### BVP
- **Filtering**: Butterworth band-pass filter of order 3 with cutoff frequencies 0.7 Hz and 3.7 Hz.

#### TEMP
- **Smoothing**: Savitzky–Golay filter with window size 11 and order 3.

#### ACC
- **Filtering**: Finite Impulse Response (FIR) filter with a length of 64 with a cut-off frequency of 0.4 Hz.


In [ ]:
# set config_files
CHEST_CONFIG = 'config_files/dataset/wesad_chest_configuration.json'
WRIST_CONFIG = 'config_files/dataset/wesad_wrist_configuration.json'

In [ ]:
from src.ml_pipeline.preprocessing import SignalPreprocessor

# preprocess the chest data
signal_preprocessor = SignalPreprocessor('src/wesad/WESAD/raw/merged_chest.pkl', 'src/wesad/WESAD/cleaned/chest_preprocessed.pkl', CHEST_CONFIG)
signal_preprocessor.preprocess_signals()

# preprocess the wrist data
signal_preprocessor = SignalPreprocessor('src/wesad/WESAD/raw/merged_wrist.pkl', 'src/wesad/WESAD/cleaned/wrist_preprocessed.pkl', WRIST_CONFIG, wrist=True)
signal_preprocessor.preprocess_signals()

## Traditional Machine Learning: Manual Feature Extraction

During the feature extraction, data is loaded in an augmented manner using a 60-second window with a sliding length of 5 seconds.

The manual feature extraction derives the following features:

In [ ]:
from src.ml_pipeline.data_loader import DataAugmenter
from src.ml_pipeline.feature_extraction import ManualFE

WINDOW_LENGTH = 60
SLIDING_LENGTH = 5

wrist_augmenter = DataAugmenter('src/wesad/WESAD/cleaned/wrist_preprocessed.pkl', WRIST_CONFIG)
batches = wrist_augmenter.segment_data(WINDOW_LENGTH, SLIDING_LENGTH)

manual_fe = ManualFE(batches, 'src/wesad/WESAD/manual_fe/wrist_manual_fe.hdf5', WRIST_CONFIG)
manual_fe.extract_features()

In [ ]:
from src.ml_pipeline.data_loader import DataAugmenter
from src.ml_pipeline.feature_extraction import ManualFE

CHEST_CONFIG = 'config_files/dataset/wesad_chest_configuration.json'
WRIST_CONFIG = 'config_files/dataset/wesad_wrist_configuration.json'

chest_augmenter = DataAugmenter('src/wesad/WESAD/cleaned/chest_preprocessed.pkl', CHEST_CONFIG)
batches = chest_augmenter.segment_data()

manual_fe = ManualFE(batches, 'src/wesad/WESAD/manual_fe/chest_manual_fe.hdf5', CHEST_CONFIG)
manual_fe.extract_features()

## Traditional Machine Learning: Automatic Feature Extraction

The automatic feature extraction uses autoencoders to derive features from its latent space:

Now, using the preprocessed `.pkl` files we will make it into a dataloader - where LOSOCV (Leave one subject out cross validation). The data augmented samples will be used in the training set but ignored in the test set.

In [1]:
from src.ml_pipeline.data_loader import LOSOCVDataLoader
from src.ml_pipeline.train import H2OTrainer

# Set Parameters
params = {
    'batch_size': 32,
    'shuffle': True,
    # 'num_workers': 4
}

CHEST_CONFIG = 'config_files/dataset/wesad_chest_configuration.json'
WRIST_CONFIG = 'config_files/dataset/wesad_wrist_configuration.json'

losocv_loader = LOSOCVDataLoader('src/wesad/WESAD/manual_fe/wrist_manual_fe.hdf5', WRIST_CONFIG, **params)
dataloaders = losocv_loader.get_data_loaders()

# Load tradtional model config
TRADTIONAL_ML_CONFIG = 'config_files/model_training/traditional_models.json'

for i, (subject_id, loaders) in enumerate(dataloaders.items()):
    train_loader = loaders['train']
    val_loader = loaders['val']
    
    print(f'Fold {i}')
    print(f'Train: {len(train_loader.dataset)}')
    print(f'Val: {len(val_loader.dataset)}')
    print()

    # Initialize trainer
    trainer = H2OTrainer(TRADTIONAL_ML_CONFIG, train_loader, val_loader, 'label')

    trainer.train()

    # Example of predicting on validation data
    predictions = trainer.predict(val_loader)
    print(predictions)

    # Shutdown H2O cluster
    trainer.shutdown()


Fold 0
Train: 49640
Val: 288

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "17.0.3-internal" 2022-04-19; OpenJDK Runtime Environment (build 17.0.3-internal+0-adhoc..src); OpenJDK 64-Bit Server VM (build 17.0.3-internal+0-adhoc..src, mixed mode, sharing)
  Starting server from /home/fsociety/anaconda3/envs/ML_Dev_C11/lib/python3.12/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpptd2swra
  JVM stdout: /tmp/tmpptd2swra/h2o_fsociety_started_from_python.out
  JVM stderr: /tmp/tmpptd2swra/h2o_fsociety_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Europe/London
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.2
H2O_cluster_version_age:,19 days
H2O_cluster_name:,H2O_from_python_fsociety_kr271m
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.817 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


[[ 1.16176471e+03  2.09243621e+03             nan             nan
              nan             nan             nan             nan
   2.52490137e+03  2.55047858e+03  1.80108433e+00  2.17333283e+00
   7.81250000e+02  4.16981250e+02  5.33736000e-01  5.31250000e+02
   8.28719977e-01  4.84375000e+02  1.09375000e+03  8.62745098e+01
   9.01960784e+01  3.12500000e+02  1.51406250e+04  1.70000000e+01
   1.25000000e+02             nan             nan  6.67743532e-03
   2.51338869e-04  3.45747232e-06  6.93223167e-03  2.65674599e+01
   9.63244687e-01  3.62565594e-02 -8.28870845e+00  1.80346070e+03
   2.38124452e+03  7.57360569e-01  1.34915097e+07  1.32037505e+00
   7.83703066e+00  1.25765434e+04  5.68627451e-01  5.91836735e-01
   8.62068966e-01  1.66666667e-01  4.89461358e+01  4.94375775e+01
   4.50584378e+01  4.48979592e+01  3.79632452e-01  6.20367548e-01
   1.11121488e+03  1.42049896e+03  6.14476604e-01  3.85523396e-01
   1.86662324e+03  1.47852735e+03  5.28870549e-01  4.71129451e-01
   1.53607

IndexError: Index (35) out of range for (0-0)

In [ ]:
from src.ml_pipeline.data_loader import LOSOCVDataLoader

# Set Parameters
params = {
    'batch_size': 32,
    'shuffle': True,
    # 'num_workers': 4
}

CHEST_CONFIG = 'config_files/dataset/wesad_chest_configuration.json'
WRIST_CONFIG = 'config_files/dataset/wesad_wrist_configuration.json'

losocv_loader = LOSOCVDataLoader('src/wesad/WESAD/manual_fe/chest_manual_fe.hdf5', CHEST_CONFIG, **params)
dataloaders = losocv_loader.get_data_loaders()

for i, (subject_id, loaders) in enumerate(dataloaders.items()):
    train_loader = loaders['train']
    val_loader = loaders['val']
    
    print(f'Fold {i}')
    print(f'Train: {len(train_loader.dataset)}')
    print(f'Val: {len(val_loader.dataset)}')
    print()
